<a href="https://colab.research.google.com/github/marco-siino/DA-ESWA/blob/main/code/augmentation/ISS_TestSet_Augmentation_IT_DE_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing modules.

In [1]:
import os
import os.path
from os import path
import re
import shutil
import string
import tensorflow as tf

!pip install -U deep-translator
from deep_translator import GoogleTranslator
from io import open
from pathlib import Path
from google.colab import files

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 4.4 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
!pip install xmltodict
import xmltodict 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importing DS and extract in current working directory.

In [3]:
# Url obtained starting from this: https://drive.google.com/file/d/19ZcqEv88euKB71HfAWjTGN3uCKp2qsfP/ and forcing export=download.
train_set_url = 'https://github.com/marco-siino/DA-ESWA/raw/main/data/iss/iss-test-original.zip'  # train FNS
train_set_path = tf.keras.utils.get_file("pan22-author-profiling-test-2022-04-22-without_truth.zip", train_set_url,
                                    extract=True, archive_format='zip',cache_dir='.',
                                    cache_subdir='')

train_set_dir = os.path.join(os.path.dirname(train_set_path), 'pan22-author-profiling-test-2022-04-22-without_truth.zip')

print(train_set_path)
print(train_set_dir)

2446477/2446477 [==============================] - 0s 0us/step
./pan22-author-profiling-test-2022-04-22-without_truth.zip
./pan22-author-profiling-test-2022-04-22-without_truth.zip


## Function to pre-process source text.

In [4]:
def custom_standardization(text):
  tag_open_CDATA_removed = re.sub('<\!\[CDATA\[', ' ',text)
  tag_closed_CDATA_removed = re.sub('\]{1,}>', ' ',tag_open_CDATA_removed)
  tag_author_lang_en_removed = re.sub('<author lang="en">', ' ',tag_closed_CDATA_removed)
  tag_author_lang_en_removed = re.sub('<author lang="en"', ' ',tag_closed_CDATA_removed)
  #only for test remove if needed
  tag_author_lang_en_removed = re.sub('class="0">', ' ', tag_author_lang_en_removed )
  tag_author_lang_en_removed = re.sub('class="1">', ' ', tag_author_lang_en_removed )
  ######
  tag_closed_author_removed = re.sub('</author>', '', tag_author_lang_en_removed)
  tag_closed_author_removed = re.sub('</author', '', tag_closed_author_removed)
  tag_open_documents_removed = re.sub('<documents>\n(\t){0,2}', '',tag_closed_author_removed)
  tag_closed_documents_removed = re.sub('</documents>\n(\t){0,2}', ' ',tag_open_documents_removed)
  lowercased_text = tag_closed_documents_removed.lower()
  return lowercased_text

## Data Augmentation function.

In [5]:
def chunkstring(string, length):
  res = list((string[0+i:length+i] for i in range(0, len(string), length)))
  return res

def enhance_one_sample(sample, TARGET='it', return_both=True):  
  preprocessed_text = custom_standardization(sample)

  #chunk to avoid character limits  
  TOBETRANS = chunkstring(preprocessed_text, 4000)
  translated_it = GoogleTranslator(source='en', target=TARGET).translate_batch(TOBETRANS)
  reversed_trans = GoogleTranslator(source=TARGET, target='en').translate_batch(translated_it)
  merged_chunks =''.join(reversed_trans)
  enhanced_sample = preprocessed_text+merged_chunks
  if return_both == False:
    enhanced_sample = merged_chunks
  return enhanced_sample

# Generating augmented train set.

In [6]:
## Generate the augmented training set as a zip file.

# pan22-author-profiling-training-2022-03-29-augmented.zip has to be created.
if not os.path.exists('pan22-author-profiling-test-2022-04-22-without_truth-augmented'):
    os.makedirs('pan22-author-profiling-test-2022-04-22-without_truth-augmented')
if not os.path.exists('pan22-author-profiling-test-2022-04-22-without_truth-augmented/en/'):
    os.makedirs('pan22-author-profiling-test-2022-04-22-without_truth-augmented/en/')

In [7]:
# Copy the ground truth file.
if os.path.exists('/content/pan22-author-profiling-test-2022-04-22-without_truth'):
        shutil.copyfile('/content/pan22-author-profiling-test-2022-04-22-without_truth/truth.txt', '/content/pan22-author-profiling-test-2022-04-22-without_truth-augmented/truth.txt')

In [8]:
file_nr = 0
count_errors = 0
for filename in os.listdir('pan22-author-profiling-test-2022-04-22-without_truth/en/'):
  file_nr += 1
  x = filename.split(".")      
  author_id = x[0]
  print("File nr.:", file_nr)
  print("Filename:", filename)
  text = open('pan22-author-profiling-test-2022-04-22-without_truth/en/'+filename, 'r').read()
  try:
    enhanced_sample = enhance_one_sample(text, TARGET='de')
    de_back = enhance_one_sample(text, TARGET='it', return_both=False)
    enhanced_sample = enhanced_sample + de_back    
    f = open("pan22-author-profiling-test-2022-04-22-without_truth/en/"+author_id+".xml", "a")
    f.write(enhanced_sample)
    f.close()
    print('Succes')
  except:
    print('!!! FAILED !!!!')
    count_errors += 1
    f = open("pan22-author-profiling-test-2022-04-22-without_truth/en/"+author_id+".xml", "a")
    preprocessed_text = custom_standardization(text)
    f.write(preprocessed_text)
    f.close()

print('cunt err', count_errors)
!zip -r pan22-author-profiling-test-2022-04-22-without_truth-augmented.zip pan22-author-profiling-test-2022-04-22-without_truth-augmented
# If automatic download doesn't start, open the directory browser on the left menu and download the zip file manually.
files.download("pan22-author-profiling-test-2022-04-22-without_truth-augmented.zip")

File nr.: 1
Filename: 740c19c02e8aefb4ee35febedd3777a2.xml
Succes
File nr.: 2
Filename: e59cae77d833f47f5673f0628cbfb69.xml
Succes
File nr.: 3
Filename: a46e5a901b594c2161de8b352a45b3e3.xml
Succes
File nr.: 4
Filename: 3a439aff76fd834539ebf05db59e948.xml
Succes
File nr.: 5
Filename: ebc12c72878213b4454d2f6ccdb708d.xml
Succes
File nr.: 6
Filename: c3c06e0097a689a9df6f4a93fa01133b.xml
Succes
File nr.: 7
Filename: c1affe96dec0745b07886970920b611e.xml
Succes
File nr.: 8
Filename: 816fc853eb4164035f1c948cf4951887.xml
Succes
File nr.: 9
Filename: bb06a89aa2a4034225d7f5143a6f4c8d.xml
Succes
File nr.: 10
Filename: fbe0f8f92e609789a5d968f01787b8ca.xml
Succes
File nr.: 11
Filename: d0c79ce811617b4677cb8b2f7985ea7f.xml
Succes
File nr.: 12
Filename: 34a758e10454a74db3c8654a96e836c9.xml
Succes
File nr.: 13
Filename: d6e240638b048e7c65ec3b1c6c3f4e7c.xml
Succes
File nr.: 14
Filename: e78c14a264bf00eb3517b17df72b7f06.xml
Succes
File nr.: 15
Filename: dbb395e9db8c0578832077a3df2f5816.xml
Succes
File nr

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>